<img src="https://datasciencedegree.wisconsin.edu/wp-content/themes/data-gulp/images/logo.svg" width="300">


Aretha Miller
DS:710 Final Project
May 3, 2020
# Lesson 13 Activity -- Using ```tweepy``` for Data Mining
Topic: Analysis of Four Types of Masks Tweeted During the Covid-19 Pandemic

This is an introduction to data collection from <a href="http://www.twitter.com/">Twitter</a> using the [`tweepy`](http://www.tweepy.org/) package.

---

## Getting set up -- things you do about once

### Install Tweepy

You must install the tweepy package from either Anaconda or the terminal on your computer before using it!

### Make a twitter account

You'll need to set up an app at <a href="https://apps.twitter.com/">apps.twitter.com</a>. #done

### Save your credentials to an external file

Make a plain text file on your computer called `twitter_credentials.py`, and put it anywhere but this directory.  I put mine in my home directory for my user.  It will look something like this:

    con_key = 'your consumer key goes here'
    con_secret = 'your consumer secret goes here'
    acc_token = 'your access token goes here'
    acc_secret = 'your access secret goes here'
    
* Save your consumer key, consumer secret, access token, and access secret there.
* Don't share these secrets with others!  
* It's also possible to generate access tokens and secrets from within an app, but now's not the right time for this.

---

## Preliminaries to using tweepy -- things you do once per session

You have to do these things about once per session.  If you close your notebook, or restart the kernel, then you have to do these things before you can again use the Tweepy interface to the Twitter API.

#### 1. Gain access to the Tweepy library

As you would any other Python library, `import`.

In [1]:
import tweepy
import pandas as pd #to load csv
import numpy as np #to do some math later
from textblob import TextBlob 
import nltk
import json
import time
import csv
import datetime
import pathlib
import unicodedata
from toolz.curried import filter, pipe, take
import datetime
import pathlib
import unicodedata

##from matplotlib import pyplot as plt##delete b4 submission


#### 2. Load your credentials from the external file

Invoke a python plain text source file located somewhere else on your computer.

In [2]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key="chh0YzH4CEcug8ayB1oOanjI4", consumer_secret="3gtBX9EXGOY3wMQbGSmolHQJCxmX7L6rXzLYNXWhfQi1AUNdql")
auth.set_access_token("1249233874695012353-1KpxU7vFdJFe375urApKvUs10v1IHO"
, "tQr48Z3aZLgJxK4rgSqtsQ5iYyrF7s9UTiYjMJ5lR45HB") #***delete before submission

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)


🔐 If you need to check whether the four variables, such as `con_key` have the correct value, insert a cell and print the value, then delete the cell.  Keep your credentials secret and safe!!!  

In [3]:
import sys
sys.version

'3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]'

In [4]:
import os
os.getcwd()

'C:\\Users\\Chris\\Documents\\GitHub\\ds710spring2020finalproject'

#### 3. Make an `API` object

The `tweepy.API` object handles construction of the Twitter API calls for you.  It's a convenience layer, but it's really dang convenient!

--- 

## Using the API

Twitter has two versions of its API:
* The [REST](https://en.wikipedia.org/wiki/Representational_state_transfer) [API](https://en.wikipedia.org/wiki/Application_programming_interface) allows you to _pull_ information from Twitter, or _push_ information back to Twitter.  For example,  
  💡 if I wanted to have a Python script that ran as a CRON job to automatically tweet for me under certain conditions, I would use the REST API.
* The Streaming API allows us to monitor Twitter in real time, grabbing tweets as they are made.  For example,  
  💡 if I wanted to make a little device powered by a Raspberry Pi that showed interesting tweets in real time on a tiny screen by my desk, I would use the streaming API.

### Method 1. The REST API

The REST API allows you to _pull_ information from Twitter, or _push_ information back to Twitter.  We'll use the REST API to run a specific search.  You could also use the REST API to make automatic tweets on Twitter, or get information about specific users.

In [20]:
#REST API used for a static search

output_path = "tweets1.csv"

n = 10000

query = "(mask OR n95) (#covid) -filter:replies -filter:retweets"

cursor_args = {
    "q":  query,
    "wait_on_rate_limit": True,
    "retry_count": 10,
    "retry_delay": 300,
    "wait_on_rate_limit_notify": 10,
    "until": (datetime.date.today() - datetime.timedelta(days=6)).isoformat(),
    "tweet_mode": "extended"
}

fieldnames = ["id", "lang", "geo", "created_at", "retweet_count", "favorite_count", "full_text"]

if pathlib.Path(output_path).exists():
    since_id = pd.read_csv(output_path).id.max()
    if not np.isnan(since_id):
        cursor_args["since_id"] = since_id + 1
        if "until" in cursor_args:
            del cursor_args["until"]

else:
    with open(output_path, "w", encoding="utf-8", newline='') as fw:
        writer = csv.DictWriter(fw, fieldnames=fieldnames)
        writer.writeheader()

with open(output_path, "a", encoding="utf-8", newline='') as fw:
    writer = csv.DictWriter(fw, fieldnames=fieldnames)
    statuses = pipe(
        tweepy.Cursor(api.search, **cursor_args).items(),
        filter(lambda status: status.lang == "en" and (
            "cloth" in status.full_text or "surgical" in status.full_text 
        )),
        take(n)
    )

    for status in statuses:
        row = {
            field: getattr(status, field) if field != "full_text"
            else  unicodedata.normalize("NFKD", getattr(status, field)).replace("\n", " ")
            for field in fieldnames
        }
        writer.writerow(row)


In [21]:
tweet_list1=api.search(q="mask")

In [22]:
tweet_list2 = api.search(q='n95') 

In [49]:
tweet_list1[0]

100

See [twitter's search documentation](https://dev.twitter.com/rest/public/search) for examples of query operators.  Pay attention to how to URL encode your query.  [This w3schools page](https://www.w3schools.com/tags/ref_urlencode.asp) has information on what `%23` and other encodings for URL's mean.

We retrieve a SearchResult object for each tweet, full of data such as the language, the identity of the poster, etc.

In [24]:
tweet_list2[0]

Status(_api=<tweepy.api.API object at 0x000001639D64D3C8>, _json={'created_at': 'Mon May 04 06:01:03 +0000 2020', 'id': 1257188472357564417, 'id_str': '1257188472357564417', 'text': 'RT @puma_chairo: Los cubrebocas N95 están indicados para uso exclusivo de quienes están en contacto directo con las vías respiratorias de u…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'puma_chairo', 'name': 'Puma Chairo', 'id': 1097573435587194882, 'id_str': '1097573435587194882', 'indices': [3, 15]}], 'urls': []}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2504689430, 'id_str': '2504689430', 'name': 'Guerrero Dann 👨🏽\u200d💼 🦸🏽\u200d♂️🥾🕶', 'screen_name': 'mac

In [48]:
#We can use the dir command to view a list of the attributes of each tweet
dir(tweet_list[0]) #mask

In [44]:
#Let's display the text of each tweet we found.
[tweet.text for tweet in tweet_list]

["RT @Ellethevillain: Y'all, the mask goes OVER your nose.",
 'RT @Sherni_15: 👉 Say no to handshake , adopt namaste .\n👉 Wear mask when u go outside .\n👉 Keep it home nd surrounding clean regularly .\n👉 Wa…',
 'RT @USMCSDI: Some asshole in Walmart asked me why I was going down the wrong side of the aisle and why wasn\'t I wearing a mask\n\nI said "bec…',
 'RT @sanjayjavin: #BiharFightsCorona continuing   outcome of the door to door survey for the period 16-3/5/2020 in all 4+34 districts.please…',
 'RT @rosette_thake: “Joseph Muscat applied his charm and appeal to get other countries to get migrants off his back. But when his mask fell…',
 "RT @Ellethevillain: Y'all, the mask goes OVER your nose.",
 '@NozukoPikoli True, the rationale for wearing a mask has been misunderstood. Education is also important in this regard.',
 "RT @Ellethevillain: Y'all, the mask goes OVER your nose.",
 'RT @snehmeet61: #LetsStaySafe\n@derasachasauda urges everyone to follow the guidelines of the government 

By default, the REST API returns 15 tweets.  We can get up to 100 by using the argument "count".

In [29]:
tweet_list1 = api.search(q='mask', count = 100)
len(tweet_list1)

100

In [30]:
tweet_list2 = api.search(q='n95', count = 100)
len(tweet_list2)

100

If we want more than 100 tweets, we can use a *while* loop.  The max_id argument lets us collect tweets that are older than a particular tweet index (in this case, the oldest tweet we've seen so far).

The `try/except/else` structure lets us fail gracefully in case the API search returns an error (e.g., if we run up against Twitter's rate limits).

In [34]:
#[t.text for t in tweet_list2]

['RT @PressSec: President @realDonaldTrump has led the greatest mobilization since WWII:\n\n➡️75M N95 masks delivered\n\n📈3x more than typically…',
 'Mascarillas N95  $1,50\n0999219361 en Quito, Ecuador https://t.co/aKt41JTdID',
 'RT @PressSec: President @realDonaldTrump has led the greatest mobilization since WWII:\n\n➡️75M N95 masks delivered\n\n📈3x more than typically…',
 'RT @anandrai177: This kind of N95 masks supplied to health workers,fungus coated mask,packaging of PPE and mask is more important,storage a…',
 'RT @juandipino: Repitan conmigo:\n\nImprimir N95 en mi mascarilla y tener registro Sanitario, NO HACE que NIOSH la apruebe.\n\nDejemos de falsi…',
 'RT @Vincent198964: 刚才与一位武汉的医生通电话，武汉的医院现在不敢开中央空调，气温已经突破33摄氏度。医护人员还要穿防护服戴N95口罩上班，病房像火山口一样难受。。。\n什么时候，共产党都是这样无人性的折磨人，只是再过几个星期，武汉进入夏天，恐怕谁也坚持不了',
 '↓\nＡさん大慌てで私に謝罪、連絡途切れる\n↓\nどうやら現在夫婦間で大揉めしてるらしく、配偶者と思しきアカウントから直接DM爆撃が来る様になる←今ｺｺ\n\nN95を5枚重ねで100メートルダッシュとかさせてみようかしら。\n\nしかし、何と言うか、こう、民度って……。',
 'RT @PressSec: President @realDonaldTrump h

Note that the free REST API restricts the number of tweets you can retrieve, and the dates: you may not be able to retrieve tweets that are more than a week old.  Pay attention to this restriction as you approach your final project topic!

In [ ]:
#save without indexes to be used in R
df.to_csv('tweets.csv',)

## Method 2. The Streaming API  

The Streaming API allows us to monitor Twitter in real time, grabbing tweets as they are made.

The ```tweepy``` package includes a class called ```StreamListener``` which monitors Twitter for us.  However, by default StreamListener does nothing with the tweets it collects.

In this demonstration, we'll modify ```StreamListener``` to make a class that prints each tweet we're interested in to the screen.  Later, you may wish to create your own class which saves information from tweets to a file.

In [ ]:
#Conduct exploration from streaming API (randomized sampling method).
#Create a subclass of tweepy.StreamListener to add a response to on_status

class PrintingStreamListener(tweepy.StreamListener):
    tweet_number= 0   
    def on_status(self, status):
        print(status.text)
        max_tweets=self.tweet_number
        self.tweet_number+=1 
        try: #except loop 
            #print (status)
            with open("streamingAPI.json","a") as saveFile:
                saveFile.write(json.dumps(status._json))
                saveFile.write('\n')
                
            return True
        except BaseException as e:#in case of 404 error or internet signal drops 
            print ("rate limited,",str(e))
            time.sleep(5)
    #disconnect the stream if we receive an error message indicating we are overloading Twitter
    
    def on_error(self, status_code):
                if status_code == 420: #returning False in on_data disconnects the stream
                   return False
   

Once we have created our subclass, we can set up our own Twitter stream.

In [ ]:
    #We create and authenticate an instance of our new ```PrintingStreamListener``` class
my_stream_listener = PrintingStreamListener()
my_stream = tweepy.Stream(auth = api.auth, listener=my_stream_listener)

In [ ]:
# Now, we're ready to start streaming!  
# You can pause the display of tweets by interrupting the Python kernel (use the menu bar at the top)

my_stream.filter(track=['mask', 'n95']) 


In [ ]:
# Even if you pause the display of tweets, your stream is still connected to Twitter!
# To disconnect (for example, if you want to change which words you are searching for), 

my_stream.disconnect()

In [ ]:
#exclude retweets for mask
q='mask -filter:retweets'


In [ ]:
#exclude retweets for n95
q='n95 -filter:retweets'

We'll use the ```track``` command to look for tweets with a specific keyword.  You can read more about constructing searches with ```track``` in the <a href="https://dev.twitter.com/streaming/overview/request-parameters#track">Twitter streaming API documentation</a>.

In [ ]:
#Saving for R
#save restAPI as dataframe

data = []
i = 0
with open('restAPI.json') as fp:
    for line in fp:
        line = json.loads(line)
        data.append(line)
    # read_json    does not flatten  
df = pd.json_normalize(data)

In [ ]:
#Saving for R
#save streamingAPI as dataframe

data = []
i = 0
with open('streamingAPI.json') as fp:
    for line in fp:
        line = json.loads(line)
        data.append(line)
        # read_json    does not flatten  
df = pd.json_normalize(data)

In [ ]:
#---------------------------#data for streamingAPI

In [ ]:
df.to_csv('streamingAPI.csv')

In [ ]:
df = pd.read_csv("streamingAPI.csv") 
df.head() #warning is due to mixed datatype

In [ ]:
df.columns.values #view column values

In [ ]:
df['user.description']

In [ ]:
df['user.name'] #view username

In [ ]:
df['user.location'] #view location of tweets

In [ ]:
df['text'].head(10) #text of tweets


In [ ]:
#sentiment analysis for masks and N95
from textblob import TextBlob

feelings = TextBlob("string")
if feelings.sentiment.polarity<0:
    sentiment="negative"
elif feelings.sentiment.polarity==0:
    sentiment="neutral"
else: 
    sentiment="positive"
    
print("Sentiment: ", sentiment)

In [ ]:
#----------------------#data for restAPI below

In [ ]:
df.to_csv('restAPI.csv') 

In [ ]:
df = pd.read_csv("restAPI.csv")
df.head()#warning is due to mixed datatype

In [ ]:
df.columns.values #view column values

In [ ]:
df['user.name'] #username

In [ ]:
df['text'].head(100) #text of tweets

In [ ]:
#sentiment analysis for restAPI

from textblob import TextBlob

feelings = TextBlob("string")
if feelings.sentiment.polarity<0:
    sentiment="negative"
elif feelings.sentiment.polarity==0:
    sentiment="neutral"
else: 
    sentiment="positive"
    
print("Sentiment: ", sentiment)

#FYI:Sentiment from restAPI (7 days ago) and streamingAPI were both neutral

In [ ]:
A .csv or .txt file containing the data from 100 tweets, 
#along with any variables you computed about the tweets 
    #(such as sentiment score or number of exclamation points). 

#?df.to_csv(C:\Users\Chris\Documents\GitHub\ds710spring2020finalproject, tweet_list[100],index = False)

In [ ]:
##remove credentials before Github submission

---

## Suggestions for skills to learn

* Collect 1000 tweets matching a search, or all available in the current time window, whichever comes first.  That 1000 was arbitrary
* Extract just the fields you are most interested in from a search, and create a Pandas data frame
* Follow the graph of followers from a specific Twitter user

---

## Useful resources and links

* [the structure of the Status object of Tweepy](https://gist.github.com/dev-techmoe/ef676cdd03ac47ac503e856282077bf2)
* [Tweet Data Dictionary](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)
* [Standard Operators](https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators) -- premium operators cost money.
* [Twitter operators by product](https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/operators-by-product) -- by product they mean *paid access level*
* [How to use Twitter’s Search REST API most effectively](https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively./)
* [Collecting Tweets with Tweepy](http://www.dealingdata.net/2016/07/23/PoGo-Series-Tweepy/)
